# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.126015e+02     1.899871e+00
 * time: 0.3677330017089844
     1     1.037079e+01     1.090411e+00
 * time: 2.3444690704345703
     2    -1.246678e+01     1.279433e+00
 * time: 3.0169551372528076
     3    -3.438502e+01     9.656248e-01
 * time: 4.007810115814209
     4    -4.786761e+01     6.413934e-01
 * time: 5.017750024795532
     5    -5.705687e+01     2.336640e-01
 * time: 5.994978189468384
     6    -5.980880e+01     1.470664e-01
 * time: 6.65503716468811
     7    -6.087065e+01     4.670826e-02
 * time: 7.3188111782073975
     8    -6.129091e+01     7.536078e-02
 * time: 8.140619039535522
     9    -6.156683e+01     4.336126e-02
 * time: 8.812929153442383
    10    -6.176473e+01     2.596315e-02
 * time: 9.489200115203857
    11    -6.190852e+01     1.933111e-02
 * time: 10.1636381149292
    12    -6.197245e+01     1.769442e-02
 * time: 10.836238145828247
    13    -6.204786e+01     1.669765e-02
 * time: 11.852210998535156
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671069
    AtomicLocal         -18.8557708
    AtomicNonlocal      14.8522669
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485382 
    Xc                  -19.3336824

    total               -62.261666460413
